In [2]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

In [5]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 10.1 MB/s eta 0:00:00


In [6]:
!wandb off

/bin/bash: line 1: wandb: command not found


In [7]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

from IPython.display import display, Image


Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.4/201.2 GB disk)


In [17]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn as nn
import json
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

class maskedFaceTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']
        self.images = []
        self.labels = []

        for filename in os.listdir(root_dir):
            if filename.endswith(".xml"):
                tree = ET.parse(os.path.join(root_dir, filename))
                root = tree.getroot()
                image_path = os.path.join(root_dir, root.find('filename').text)
                self.images.append(image_path)

                label_counts = [0, 0, 0]
                for obj in root.findall('object'):
                    name = obj.find('name').text
                    if name == 'with_mask':
                        label_counts[0] += 1
                    elif name == 'without_mask':
                        label_counts[1] += 1
                    elif name == 'mask_weared_incorrect':
                        label_counts[2] += 1

                self.labels.append(label_counts)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]

        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label

# Example usage
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

test = maskedFaceTestDataset(root_dir='MaskedFace/val', transform=transform)
test_data_loader = DataLoader(test, batch_size=1, shuffle=True)

In [36]:
import numpy as np

def count_masks(dataset, model):
    total_images = len(dataset)
    total_classes = 3
    total_counts = np.zeros((total_images, total_classes), dtype=np.int64)
    total_mape = 0.0

    for idx, (inputs, targets) in enumerate(dataset):
        results = model(inputs)
        results = results[0].tojson()
        results = json.loads(results)
        predicted = [0, 0, 0]
        actual = [0, 0, 0]
        for i, target in enumerate(targets):
            actual[i] = int(target[0])
        for result in results:
            predicted[result['class']] += 1

        total_counts[idx] = predicted
        true_counts = np.array(actual)
        abs_diff = np.abs(true_counts - predicted)
        mape = np.mean(np.divide(abs_diff, np.maximum(true_counts, 1)) * 100)
        total_mape += mape

    avg_mape = total_mape / total_images
    return total_counts, avg_mape

#the model i copied in the data folder
model = YOLO('data/task_7.pt', verbose=False)
counts, mape = count_masks(test_data_loader, model)
print("Counts:")
print(counts)
print("Average MAPE:", mape)



0: 640x640 7 with_masks, 2 without_masks, 15.7ms
Speed: 1.2ms preprocess, 15.7ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 with_masks, 15.8ms
Speed: 1.1ms preprocess, 15.8ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 with_mask, 15.6ms
Speed: 1.1ms preprocess, 15.6ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 with_mask, 15.7ms
Speed: 1.1ms preprocess, 15.7ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 55 with_masks, 15.8ms
Speed: 1.2ms preprocess, 15.8ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 with_masks, 1 without_mask, 15.8ms
Speed: 1.2ms preprocess, 15.8ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 with_mask, 15.8ms
Speed: 1.1ms preprocess, 15.8ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 with_masks, 3 without_masks, 15.8ms
Speed: 1.